In [13]:
from datetime import datetime, timedelta
import pandas as pd



click_stream = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/click_stream_final.csv')
transaction = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/transaction_final.csv')
product = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/product_final.csv')
customer = pd.read_csv('C:/Users/hm981/OneDrive/바탕 화면/AI부트캠프/project/teamproject2/data_refine/customer_final.csv')

tem = pd.merge(click_stream[['session_id', 'event_time', 'event_name']], transaction[['session_id', 'customer_id']].drop_duplicates(), on = 'session_id', how = "left")
tem2 = pd.merge(tem[tem.customer_id.notnull()], customer[['customer_id', 'first_join_date']], on = 'customer_id', how= 'left')




In [56]:
#함수 사용에 필요한 특성
#click_stream
#   event_name: 'HOMEPAGE' 인 이벤트만 조회하기 때문에
#   event_time: 재방문에 걸린 시간 조회하기 위해
#   session_id: transaction 테이블과 병합하여 customer_id를 얻기 위해
#transaction
#   session_id: click_stream 테이블과 병합하여 customer_id를 얻기 위해
#   customer_id: customer 테이블과 병합하여 first_join_date 를 얻기 위해
#customer
#   customer_id: click_stream 테이블과 병합하여 first_join_date 를 얻기 위해
#   first_join_date: 가입 시기를 알기 위해


def bounce_judge(df):
    #측정 기간 설정
    start_date = pd.to_datetime('2020-07-01')
    end_date = pd.to_datetime('2022-06-30')
    
    #측정 기간에 해당하는 월 수: 24개
    num_of_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month) + 1
    
    #이탈 판단 기준(10일 = 60 * 60 * 24 * 10 = 864000)
    criterion = 864000

    # 월별 컬럼 이름 생성
    column_names = [(start_date + pd.DateOffset(months=i)).strftime('%Y-%m') for i in range(num_of_months)]

    #데이터의 시간 데이터 타입 초기화
    df.event_time = pd.to_datetime(df.event_time)

    # 빈 데이터프레임 생성
    res = pd.DataFrame(columns=column_names)

    #customer_id를 기준으로 반복
    for customer in df.customer_id.unique():
    
        #베이스 딕셔너리 생성
        tem = {month : 0 for month in sorted(column_names)}

        #조건에 맞는 데이터로그 불러오기
        log = df[(df.customer_id == customer) & (df.event_name == 'HOMEPAGE') & (pd.to_datetime(df.event_time.dt.date) <= end_date) & (pd.to_datetime(df.event_time.dt.date) >= start_date)].sort_values('event_time').reset_index(drop = True)

        log.event_time = pd.to_datetime(log.event_time)

        # 2020-07월 이전에 계정을 생성한 경우
        if pd.to_datetime(log.first_join_date.iloc[0]) < start_date:

            #방문 기록이 없는 달 추출하여 집합으로 저장
            date_set = set([date.strftime('%Y-%m') for date in log.event_time])
            bounce_month = set(column_names) - date_set
      
            #homepage 재방문 시간 계산
            log['diff'] = log.event_time.diff().dt.total_seconds()

            #첫번째 인덱스에는 start_date 부터의 시간 간격 측정
            log.iloc[0,4] = (pd.Timestamp(log.iloc[0,3], tz= 'UTC') - pd.Timestamp('2020-07-01 0:0:0', tz= 'UTC')).total_seconds()

            #bounce_month 집합에 마지막 접속일 부터 10일 지난 시점의 달 추가(합집합)
            bounce_month = bounce_month.union(set([date.strftime('%Y-%m') for date in log.loc[(log[log['diff'] > criterion].index - 1), 'event_time'] + timedelta(days=10)]))

        # 2020-07월 이후에 계정을 생성한 경우
        else:
      
            #방문 기록이 없는 달 추출하여 집합으로 저장
            date_set = set([date.strftime('%Y-%m') for date in log.event_time])

            #start_date 이후 first_join_date 이전 월 추출(이탈 아닌것으로 확인해야하기 때문에)
            date_range = pd.date_range(start=start_date, end=log.first_join_date.iloc[0], freq='M')
            extracted_months = set(date.strftime('%Y-%m') for date in date_range)
      
            bounce_month = set(column_names) - date_set - extracted_months
      
            #homepage 재방문 시간 계산
            log['diff'] = log.event_time.diff().dt.total_seconds()

            #첫번째 인덱스에는 first_join_date 부터의 시간 간격 측정
            log.iloc[0,4] = (pd.Timestamp(log.iloc[0,3], tz= 'UTC') - pd.Timestamp(log.first_join_date.iloc[0], tz= 'UTC')).total_seconds()

            #bounce_month 집합에 마지막 접속일 부터 10일 지난 시점의 달 추가(합집합)
            bounce_month = bounce_month.union(set([date.strftime('%Y-%m') for date in log.loc[(log[log['diff'] > criterion].index - 1), 'event_time'] + timedelta(days=10)]))

        #이탈(1) 입력
        for month in bounce_month:
            tem[month] = 1
    
        #customer_id 와 key에 대해 이탈여부 매칭하는 데이터프레임 만들기
        for key in tem:
            res.loc[customer, key] = tem[key]
    
    return res